# 4. V4 Data Optimization Challenge

Benchmarking efficient on-chain data retrieval from the UniV4 singleton.

**Objectives:**
1. Query all required on-chain data as efficiently as possible
2. Document approach and timing results

**Optimization Strategy:**
- **Multicall3 Batching**: Aggregate 220+ individual RPC calls into 2 multicall requests

**Data to Fetch:**
- Pool state (slot0, current liquidity)
- Tick bitmap (201 word positions)
- Tick liquidity for all initialized ticks

In [1]:
import os
import sys
from web3 import Web3
from dotenv import load_dotenv

# Add parent directory to path for imports
sys.path.append("..")

# Optimized fetch functions
from src.optimized_v4_fetch import (
    fetch_all_v4_data_sync,
    fetch_original_sequential,
)

In [2]:
load_dotenv()
url = os.getenv('rpc_url_mainnet')
w3 = Web3(Web3.HTTPProvider(url))

if not w3.is_connected():
    raise Exception('Could not connect to RPC')
else:
    print('Connected to RPC')
    print(f'Latest block: {w3.eth.block_number}')

Connected to RPC
Latest block: 23971032


## Benchmark: Original vs Optimized

### Original Approach
Individual sequential RPC calls:
- 1x `getSlot0()`
- 1x `getLiquidity()`
- 201x `getTickBitmap()` (sequential loop)
- Nx `getTickLiquidity()` (one per initialized tick)

**Total: ~220+ sequential RPC calls**

### Optimized Approach
Multicall3 batched calls:
- 1x Multicall (slot0 + liquidity + 201 bitmap queries)
- 1x Multicall (all tick liquidity queries)

**Total: 2 RPC calls**

In [3]:
# Run original sequential approach
print("Running ORIGINAL sequential approach...")
print("(This will make 220+ individual RPC calls)\n")

original_data, original_ms = fetch_original_sequential(w3, search_range=100)

print(f"Original approach completed in {original_ms:.2f} ms")
print(f"Found {len(original_data['initialized_ticks'])} initialized ticks")

Running ORIGINAL sequential approach...
(This will make 220+ individual RPC calls)

Original approach completed in 42281.94 ms
Found 20 initialized ticks


In [4]:
# Run optimized multicall approach
print("Running OPTIMIZED multicall approach...")
print("(This batches all calls into just a few RPC requests)\n")

optimized_data = fetch_all_v4_data_sync(w3, search_range=100)

print(f"Optimized approach completed in {optimized_data.timing.total_ms:.2f} ms")
print(f"Found {len(optimized_data.initialized_ticks)} initialized ticks")

Running OPTIMIZED multicall approach...
(This batches all calls into just a few RPC requests)

Optimized approach completed in 797.14 ms
Found 20 initialized ticks


## Results Comparison

In [5]:
# Calculate improvement
speedup = original_ms / optimized_data.timing.total_ms
reduction_pct = (1 - optimized_data.timing.total_ms / original_ms) * 100

print("="*60)
print("BENCHMARK RESULTS")
print("="*60)
print(f"\nOriginal (sequential):  {original_ms:,.2f} ms")
print(f"Optimized (multicall):  {optimized_data.timing.total_ms:,.2f} ms")
print(f"\nSpeedup:                {speedup:.1f}x faster")
print(f"Time reduction:         {reduction_pct:.1f}%")
print("="*60)

BENCHMARK RESULTS

Original (sequential):  42,281.94 ms
Optimized (multicall):  797.14 ms

Speedup:                53.0x faster
Time reduction:         98.1%


In [6]:
# Detailed timing breakdown for optimized approach
t = optimized_data.timing

print("\nOPTIMIZED APPROACH - TIMING BREAKDOWN")
print("-"*50)
print(f"Phase 1 (bitmap + state multicall): {t.phase1_bitmap_ms:,.2f} ms")
print(f"  - Batched {t.num_bitmap_calls + 2} calls into 1 RPC request")
print(f"Phase 2 (parse bitmaps):            {t.phase2_parse_ms:,.2f} ms")
print("  - Pure computation, no RPC")
print(f"Phase 3 (tick liquidity multicall): {t.phase3_tick_liquidity_ms:,.2f} ms")
print(f"  - Batched {t.num_tick_calls} calls into 1 RPC request")
print("-"*50)
print(f"TOTAL:                              {t.total_ms:,.2f} ms")


OPTIMIZED APPROACH - TIMING BREAKDOWN
--------------------------------------------------
Phase 1 (bitmap + state multicall): 627.84 ms
  - Batched 203 calls into 1 RPC request
Phase 2 (parse bitmaps):            0.10 ms
  - Pure computation, no RPC
Phase 3 (tick liquidity multicall): 169.20 ms
  - Batched 20 calls into 1 RPC request
--------------------------------------------------
TOTAL:                              797.14 ms


## Tick Liquidity Data

Comparing gross and net liquidity at each initialized tick from both methods.

In [7]:
# Compare tick liquidity from both methods
import pandas as pd

all_ticks = sorted(set(original_data['initialized_ticks']) | set(optimized_data.initialized_ticks))

comparison_data = []
for tick in all_ticks:
    orig = original_data['tick_liquidity'].get(tick, (0, 0))
    opt = optimized_data.tick_liquidity.get(tick, (0, 0))
    match = "✓" if orig == opt else "✗"
    comparison_data.append({
        'Tick': tick,
        'Gross Liq (Original)': f"{orig[0]:,}",
        'Gross Liq (Optimized)': f"{opt[0]:,}",
        'Net Liq (Original)': f"{orig[1]:,}",
        'Net Liq (Optimized)': f"{opt[1]:,}",
        'Match': match
    })

df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))
print(f"\nTotal initialized ticks: {len(all_ticks)}")
print(f"All matches: {all(row['Match'] == '✓' for row in comparison_data)}")

   Tick           Gross Liq (Original)          Gross Liq (Optimized)              Net Liq (Original)             Net Liq (Optimized) Match
-887220        104,617,165,570,455,737        104,617,165,570,455,737         104,617,165,570,455,737         104,617,165,570,455,737     ✓
  62160    252,607,173,771,615,891,305    252,607,173,771,615,891,305     252,607,173,771,615,891,305     252,607,173,771,615,891,305     ✓
  68880 40,518,051,941,909,325,704,777 40,518,051,941,909,325,704,777  40,518,051,941,909,325,704,777  40,518,051,941,909,325,704,777     ✓
  69000     22,364,597,328,919,231,573     22,364,597,328,919,231,573      22,364,597,328,919,231,573      22,364,597,328,919,231,573     ✓
  73080    141,056,643,108,128,760,485    141,056,643,108,128,760,485     141,056,643,108,128,760,485     141,056,643,108,128,760,485     ✓
  73800     51,735,960,708,368,126,295     51,735,960,708,368,126,295      51,735,960,708,368,126,295      51,735,960,708,368,126,295     ✓
  74280    159,140,3